In [1]:
import keras
from keras.layers import Input, Dense, Lambda, Flatten
from keras.layers.merge import concatenate
from keras.models import Model, Sequential
from keras.objectives import binary_crossentropy
from keras.callbacks import LearningRateScheduler

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import keras.backend as K
import tensorflow as tf

from models import vgg
from utils.angles import deg2bit, bit2deg
from utils.losses import maad_from_deg 
from utils.losses import von_mises_log_likelihood_tf
from utils.towncentre import load_towncentre
from utils.experiements import get_experiment_id

Using TensorFlow backend.


In [2]:
xtr, ytr_deg, xte, yte_deg = load_towncentre('data/TownCentre.pkl.gz', canonical_split=True)
image_height, image_width = xtr.shape[1], xtr.shape[2]
ytr_bit = deg2bit(ytr_deg)
yte_bit = deg2bit(yte_deg)

In [4]:
#x_tf = tf.placeholder(tf.float32, shape=[None, 3, image_width, image_height])
mu_pred_tf = tf.placeholder(tf.float32, shape=[None, 2])
kappa_tf = tf.placeholder(tf.float32, shape=[None, 1])
y_true_tf = tf.placeholder(tf.float32, shape=[None, 2])
y_pred_tf = tf.concat([mu_pred_tf, kappa_tf], axis=1)

In [29]:
def von_mises_neg_log_likelihood_keras(y_true, y_pred):
    mu_pred = y_pred[:,0:2]
    #kappa_pred = tf.abs(y_pred[:, 2:])
    kappa_pred = tf.ones(shape=tf.shape(y_pred[:, 2:]))
    return -von_mises_log_likelihood_tf(y_true, mu_pred, kappa_pred, input_type='biternion')

In [30]:
X = Input(shape=[50,50,3])
vgg_x = vgg.vgg_model(final_layer=False,
                      image_height=50,
                      image_width=50)(X)

mu = Lambda(lambda x: K.l2_normalize(x, axis=1))(Dense(2)(vgg_x))
kappa = Dense(1)(vgg_x) # Input(shape=[1]) 
mu_kappa = concatenate([mu, kappa])

biternion_vgg_prob = Model(X, mu_kappa)

optimizer = keras.optimizers.Adam(lr=0.0001)

biternion_vgg_prob.compile(loss=von_mises_neg_log_likelihood_keras,
                           optimizer='adam')

In [32]:
biternion_vgg_prob.fit(x=xtr , y=ytr_bit, batch_size=100, validation_split=0.1, epochs=5)

Train on 7002 samples, validate on 778 samples
Epoch 1/5
7002/7002 [==============================] - 51s - loss: 1.3311 - val_loss: 1.5654
Epoch 2/5
7002/7002 [==============================] - 54s - loss: 1.3217 - val_loss: 1.9654
Epoch 3/5
7002/7002 [==============================] - 47s - loss: 1.3519 - val_loss: 1.8916
Epoch 4/5
7002/7002 [==============================] - 47s - loss: 1.3112 - val_loss: 1.8602
Epoch 5/5
7002/7002 [==============================] - 48s - loss: 1.2954 - val_loss: 1.4726


In [33]:
mu_kappa = biternion_vgg_prob.predict(xte)
yte_preds = bit2deg(mu_kappa[:,0:2])
kappa_te_preds = mu_kappa[:,2]

In [34]:
np.abs(kappa_te_preds)

array([  1.61407232e+00,   1.85154796e-01,   6.64699078e-02,
         5.19262075e-01,   1.50489056e+00,   2.15072680e+00,
         1.66449094e+00,   3.85148287e+00,   1.58230531e+00,
         2.17554879e+00,   2.65501451e+00,   5.15794098e-01,
         1.50706291e+00,   2.55556107e-01,   6.61900759e-01,
         2.70003080e+00,   1.76681066e+00,   2.61150026e+00,
         5.11689186e-02,   3.54557037e-01,   5.82082272e-02,
         1.26826286e-01,   1.14773703e+00,   1.40096998e+00,
         2.76905465e+00,   1.01784956e+00,   3.31334352e-01,
         1.51034760e+00,   1.43190694e+00,   2.36771727e+00,
         1.03967571e+00,   5.64695597e-01,   4.11785126e-01,
         5.04935265e-01,   2.33155251e-01,   3.19530487e+00,
         9.82404709e-01,   2.78555799e+00,   3.84191561e+00,
         5.25722027e-01,   1.05366421e+00,   1.21080136e+00,
         1.02502239e+00,   2.00573587e+00,   5.47781765e-01,
         3.90562248e+00,   3.61279106e+00,   1.94862401e+00,
         2.29711699e+00,

In [35]:
loss = maad_from_deg(yte_preds, yte_deg)

mean_loss = np.mean(loss)
std_loss = np.std(loss)

print("MAAD error (test) : %f ± %f" % (mean_loss, std_loss))

MAAD error (test) : 32.973584 ± 35.545167
